In [1]:
#提取链家房源面积并排序
#method_1
import requests
import re
import pandas as pd

# 将链家前10页url保存在urlList中
def generate_url(pgNum):
    url = 'https://bj.lianjia.com/ershoufang/pg{}/'
    urlList = []
    for url_next in range(1, int(pgNum)+1):
        urlList.append(url.format(url_next))
    return urlList

#利用request和正则表达式
def get_Information1(allurl):
    info = {}       #返回数据字典
    resultList = []  #数据结果列表
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    for k in range(0, len(allurl)):  #对每页顺序进行存储
        getUrl = requests.get(allurl[k], headers=header)
        if getUrl.status_code == 200:
            setArea = re.compile(r'<div class="houseInfo".*?<span.*?<span.*?span>(.*?)平米')  #分别得到房屋面积，名字与价格并将价格转为float型便于排序，存入三个数组中
            setName = re.compile(r'<div class="houseInfo".*?region">(.*?) </a')
            setPrice = re.compile(r'<div class="followInfo".*?<div class="totalPrice".*?span>(.*?)</span')
            getName = re.findall(setName, getUrl.text)  #存储房屋名字
            getArea = [float(i) for i in re.findall(setArea, getUrl.text)]  #存储房屋面积
            getPrice = re.findall(setPrice, getUrl.text)  #存储房屋价格
            for i in range(0, len(getArea)):
                tup = (getName[i], (getArea[i]), getPrice[i]) #将每个房子的名字价格与面积组成一个元组存入
                resultList.append(tup)
            resultList.sort(key=lambda x: x[1], reverse=True) #根据resultList中每个元组第二个元素即面积进行排序
    # print(re_result)
    for j in range(0, len(resultList)):
        info.setdefault('name', []).append(resultList[j][0])  #将resultList中元素存入字典方便写入excel
        info.setdefault('area', []).append(str(resultList[j][1])+'平米')
        info.setdefault('price', []).append(str(resultList[j][2]) + '万')
    return info

def write_into_excel(info):  #写入excel
    pd_look = pd.DataFrame(info)
    pd_look.to_excel('G:\lianjia\picture\链家二手房1.xlsx', sheet_name='链家二手房1')
    
def main():
    pgNum = input("请输入生成页数：")
    allurl = generate_url(pgNum)
    info = get_Information1(allurl)
    print(info)
    write_into_excel(info)

main()

请输入生成页数：10
{'name': ['阿曼寓所', '碧水云天', '万科星园', 'UHN国际村', '国美第一城2号院', '瑞都国际南区', '国瑞城西区', '领秀新硅谷1号院', '北亚花园', '天通苑东一区', '天通苑东二区', '筑华年', '中铁花溪渡', '天通苑东二区', '西引力', '公园五号', '和谐家园一区', '龙锦苑东一区', '龙锦苑东一区', '芍药居甲2号院', '金隅万科城', '靓景明居', '佰嘉城', '三环新城8号院', '珠江峰景', '青年汇佳园', '宏盛家园', '龙锦苑六区', '地壳所', '龙泽苑东区', '保利金泉', '美丽园', '九龙花园', '上海建筑', '远洋山水', '建邦华府', '银河湾', '世安家园', '苏荷时代', '望京西园三区', '天通苑中苑', '曙光花园', '国家气象局', '天畅园', '北纬40度一期', '兆丰园', '新龙城', '时代国际嘉园', '望京西园三区', '龙跃苑四区', '新兴年代', '博雅德园', '天宁寺前街南里', '珠江绿洲', '翠林漫步', '京贸国际城西区', '瑞都公园世家南区', '北京人家', '北京新天地五期', '金顶街五区', '重兴园', '定福家园2号院', '观景园', '慧时欣园', '首开智慧社', '珠江帝景新博悦', '北街家园八区', '天通苑东一区', '京棉新城二厂', '望都新地', '西营房', '旗胜家园', '凯德锦绣', '首城国际B区', '旗胜家园', '建邦枫景', '育新花园', '润景公馆', '鸿业兴园二区', '瞰都国际', '金台北街', '北潞馨家园', '鑫兆雅园北区', '澳洲康都', '溪城家园', '西直门南大街', '方宝苑', '首城国际B区', '鑫兆丽园', '苹果园小区二区', '志新村', '京贸国际城西区', '长丰园一区', '新潮嘉园一期', '晨光家园', '梧桐人家', '石园南区', '旗胜家园', '芳星园三区', '珠江罗马嘉园东区', '紫南家园', '甘露晴苑', '群芳三园', '塔院小区', '银地家园', '蒲安东里', '金蝉南里', '太极小区', '溪城家园', '紫金新干线', '金丰园', '万科城市

In [2]:
#method_2
import requests
import re
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup

# 链家前10页url保存在urlList中
def generate_url(pgNum):
    url = 'https://bj.lianjia.com/ershoufang/pg{}/'
    urlList = []
    for url_next in range(1, int(pgNum)+1):
        urlList.append(url.format(url_next))
    return urlList

def get_Information2(allurl):
    info = {}         #返回数据结果字典
    resultList = []   #数据结果列表
    browser = webdriver.Chrome()
    for i in range(0, len(allurl)):
        browser.get(allurl[i])          #对每一页URL进行分析得到html内容
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        for j in range(0, len(soup.select('.houseInfo'))):      #提取房屋信息
            m = j + i * len(soup.select('.houseInfo'))          #列表当前位置
            resultList.append(soup.select('.houseInfo')[j].text.split('/'))     #将单个房屋信息分割为列表
            resultList[m].append(soup.select('.totalPrice')[j].text)            #添加价格信息
            resultList[m][2] = float((resultList[m][2][:-2]))                   #将面积转为float类型
    resultList.sort(key=lambda x: x[2], reverse=True)                           #根据面积排序
    print(resultList)
    for k in range(0, len(resultList)):
        info.setdefault('name', []).append(resultList[k][0])  # 将resultList中元素存入字典方便写入excel
        info.setdefault('area', []).append(str(resultList[k][2]) + '平米')
        info.setdefault('price', []).append(resultList[k][-1])
    return info

def write_into_excel(info):  #写入excel
    pd_look = pd.DataFrame(info)
    pd_look.to_excel('G:\lianjia\picture\链家二手房2.xlsx', sheet_name='链家二手房2')

        
def main():
    pgNum = input("请输入生成页数：")
    allurl = generate_url(pgNum)
    print(allurl)
    info = get_Information2(allurl)
    write_into_excel(info)


main()

请输入生成页数：10
['https://bj.lianjia.com/ershoufang/pg1/', 'https://bj.lianjia.com/ershoufang/pg2/', 'https://bj.lianjia.com/ershoufang/pg3/', 'https://bj.lianjia.com/ershoufang/pg4/', 'https://bj.lianjia.com/ershoufang/pg5/', 'https://bj.lianjia.com/ershoufang/pg6/', 'https://bj.lianjia.com/ershoufang/pg7/', 'https://bj.lianjia.com/ershoufang/pg8/', 'https://bj.lianjia.com/ershoufang/pg9/', 'https://bj.lianjia.com/ershoufang/pg10/']
[['阿曼寓所 ', '4室2厅', 175.1, '南 北', '简装', '有电梯', '1080万'], ['碧水云天 ', '3室1厅', 148.69, '南 北', '精装', '有电梯', '2030万'], ['万科星园 ', '3室1厅', 147.32, '东南', '精装', '有电梯', '1150万'], ['UHN国际村 ', '3室2厅', 145.26, '南 北', '精装', '有电梯', '1355万'], ['国美第一城2号院 ', '3室2厅', 144.76, '东 西', '精装', '有电梯', '745万'], ['瑞都国际南区 ', '3室2厅', 143.93, '南 北', '精装', '有电梯', '718万'], ['国瑞城西区 ', '2室2厅', 143.71, '南 西', '精装', '有电梯', '870万'], ['领秀新硅谷1号院 ', '3室2厅', 143.69, '东 南 北', '精装', '有电梯', '998万'], ['北亚花园 ', '3室2厅', 141.35, '南 北', '简装', '无电梯', '359万'], ['天通苑东一区 ', '3室1厅', 140.69, '东南', '简装', '有电梯', '446万']